In [1]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd
debug = 2
import numpy as np
import pandas
import time
import geopandas
import os
import re
pandas.set_option('display.max_colwidth', -1)
print('ok')

ok


# Process data for one year

## Create directory structure

In [2]:
thisyear = 2011

basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/acs5/'
yeardir = basedir + str(thisyear) + '/'
if (thisyear == 2017):
    rawdatadir = yeardir + 'rawdata/'
    metadir = yeardir + 'metadata/xls_temp/'
elif (thisyear == 2016):
    rawdatadir = yeardir + 'rawdata/data/tab4/sumfile/prod/2012thru2016/group2/'
    metadir = yeardir + 'metadata/templates/'
elif (thisyear == 2015):
    rawdatadir = yeardir + 'rawdata/group2/'
    metadir = yeardir + 'metadata/'
elif (thisyear == 2014):
    rawdatadir = yeardir + 'rawdata/tab4/sumfile/prod/2010thru2014/group2/'
    metadir = yeardir + 'metadata/seq/'
elif (thisyear == 2013):
    rawdatadir = yeardir + 'rawdata/group2/'
    metadir = yeardir + 'metadata/'
elif (thisyear == 2012):
    rawdatadir = yeardir + 'rawdata/tab4/sumfile/prod/2008thru2012/group2/'
    metadir = yeardir + 'metadata/'
elif (thisyear == 2011):
    rawdatadir = yeardir + 'rawdata/group2/'
    metadir = yeardir + 'metadata/'
elif (thisyear == 2010):
    rawdatadir = yeardir + 'rawdata/tab4/sumfile/prod/2006thru2010/group2/'
    metadir = yeardir + 'metadata/'
    
    
estimates_dir = yeardir + 'estimates/'
margin_of_error_dir = yeardir + 'margin_of_error/'
vardir = yeardir + 'variables/'

geodir = yeardir + 'geography/'

extrasdir = '/home/idies/workspace/Storage/raddick/census/extras/'

outdir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/datasets/acs5/'
outfile = outdir + 'acs5_md_{0:.0f}.csv'.format(thisyear)
metaoutfile = outdir + 'acs5_metadata_md_{0:.0f}.csv'.format(thisyear)

for thisdir in [estimates_dir, margin_of_error_dir, vardir, geodir]:#[datadir, errordir, vardir, geodir]:
    #print(thisdir)
    if not(os.path.exists(thisdir)):
        os.makedirs(thisdir)
print('Done!')
#os.chdir(rawdatadir)
#print(os.getcwd())

Done!


# Parse metadata

Let's see what columns are available for us to get

In [3]:
os.chdir(metadir)
themetafiles = [x for x in os.listdir() if 'seq' in x.lower()]
lastfilenum = len(themetafiles)
#os.chdir(metadir)

metadata_df = pandas.DataFrame()

state_codes_df = pandas.read_csv(extrasdir+'statecodes.csv')
state_codes_df = state_codes_df.set_index('STUSAB')
states = state_codes_df.index.values.tolist()
states = [x.lower() for x in states if x not in ('AS', 'GU', 'MP', 'PR', 'UM', 'VI')]
states = ['md']
#os.chdir(metadir)
metadata_df = pandas.DataFrame()

i = 1
if (thisyear == 2017):
    this_seq_metadata_cols_df = pandas.read_excel(metadir+'seq{0:.0f}.xlsx'.format(i))
elif (thisyear == 2011):
    this_seq_metadata_cols_df = pandas.read_excel(metadir+'Seq{0:04d}.xls'.format(i))
else:
    this_seq_metadata_cols_df = pandas.read_excel(metadir+'Seq{0:.0f}.xls'.format(i))
this_seq_metadata_df = this_seq_metadata_cols_df.T
this_seq_metadata_df.index.name = 'variable'
this_seq_metadata_df = this_seq_metadata_df.rename(columns={0: 'description'})
this_seq_metadata_df = this_seq_metadata_df.assign(sequence_number = i)
this_seq_metadata_df = this_seq_metadata_df.reset_index()
#this_seq_metadata_df.index.name = 'varnum'
metadata_df = pandas.concat((metadata_df, this_seq_metadata_df))

for i in range(2,lastfilenum+1):
    if ((np.mod(i,25) == 0) | (i == lastfilenum)):
        print('Parsing metadata for sequence {0:.0f}...'.format(i))
    if (thisyear == 2017):
        this_seq_metadata_cols_df = pandas.read_excel(metadir+'seq{0:.0f}.xlsx'.format(i))
    elif (thisyear == 2011):
        this_seq_metadata_cols_df = pandas.read_excel(metadir+'Seq{0:04d}.xls'.format(i))
    else:
        this_seq_metadata_cols_df = pandas.read_excel(metadir+'Seq{0:.0f}.xls'.format(i))
    this_seq_metadata_df = this_seq_metadata_cols_df.T
    this_seq_metadata_df = this_seq_metadata_df.assign(sequence_number = i)
    this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID', 'FILETYPE', 'STUSAB', 'CHARITER', 'SEQUENCE', 'LOGRECNO'], axis=0)
    this_seq_metadata_df.index.name = 'variable'
    this_seq_metadata_df = this_seq_metadata_df.rename(columns={0: 'description'})
    this_seq_metadata_df = this_seq_metadata_df.reset_index()
    metadata_df = pandas.concat((metadata_df, this_seq_metadata_df))
metadata_df = metadata_df.reset_index(drop=True)
metadata_df.index.name = 'varnum'
print('Done!')


Parsing metadata for sequence 25...
Parsing metadata for sequence 50...
Parsing metadata for sequence 75...
Parsing metadata for sequence 100...
Parsing metadata for sequence 113...
Done!


## Figure out what variables we need, add them to varlist

### The variables we actually want for this study

In [4]:
# IV: population by race; owner-occupied units; MFI; hs grad pct (age 25 and older); 
### female hoh pct; unemployment pct (age 18 and older); poverty pct; median home value;
### median home year built

varlist = ['STUSAB','LOGRECNO']    # Lookup variables (we always need these): Seq 1 in 2017
varlist += ['B01001_001']   # Total population: Seq 2 in 2017
varlist += ['B11001_001']  # Total households: Seq 36 in 2017
varlist += ['B02001_002', 'B02001_003'] # White-only and black-only population: Seq 4 in 2017

# WHITE-ONLY AND BLACK-ONLY HOUSEHOLDER
varlist += metadata_df['variable'][
    (metadata_df['description'].apply(lambda x: 'household' in str(x).lower()))
    & (
        (metadata_df['description'].apply(lambda x: 'white alone, not ' in str(x).lower()))
        | (metadata_df['description'].apply(lambda x: 'black' in str(x).lower()))
    )
    & (metadata_df['description'].apply(lambda x: str(x)[-5:] == 'Total'))
    & (metadata_df['description'].apply(lambda x: str(x)[0:25] == 'HOUSEHOLD TYPE (INCLUDING'))
    #& (metadata_df['sequence_number'].apply(lambda x: ((x >= 36) & (x <= 36))))
].tolist()

varlist += ['B25003_002'] # Owner-occupied housing units: Seq 103 in 2017
varlist += ['B19113_001'] # Median family income: Seq 64 in 2017
varlist += ['B15002_011', 'B15002_028'] # High school graduates: Seq 43 in 2017
varlist += ['B11001_006'] # Female householder no husband present: Seq 36 in 2017
varlist += ['B23025_005'] # Unemployed people age 16 or over: Seq 79 in 2017
varlist += ['B17001_002'] # Income in the past 12 months below poverty level: Seq 48 in 2017
varlist += ['B25077_001'] # MEDIAN VALUE (DOLLARS)% Owner-occupied housing units%Median value (dollars): Seq 106 in 2017
varlist += ['B25035_001'] # MEDIAN YEAR STRUCTURE BUILT% Housing units%Median year structure built: Seq 105 in 2017

# ANOTHER OPTION: UNEMPLOYED PEOPLE AGES 22 AND OLDER: Seq 76 in 2017
#varlist += metadata_df['variable'][
#    (metadata_df['description'].apply(lambda x: 'unemplo' in x.lower()))
#    & (metadata_df['description'].apply(lambda x: x[0:31] == 'SEX BY AGE BY EMPLOYMENT STATUS'))
#    & (metadata_df['variable'].apply(lambda x: re.search("[A-Z]_", x) == None))  # note: case-sensitive search for varnames
#    #& (metadata_df['sequence_number'].apply(lambda x: ((x >= 76) & (x <= 76))))    
#].tolist()

# Cross-tab variables to come back to later
#varlist += ['B11001A_006', 'B11001B_006', 'B17001A_002', 'B17001B_002']
#varlist += ['C15002A_004', 'C15002A_009', 'C15002B_004', 'C15002B_009']
#varlist += ['B19113A_001', 'B19113B_001', 'B25003A_002', 'B25003B_002']

# CHECK!
#metadata_df[metadata_df['variable'].apply(lambda x: x in varlist)]
print('ok')


ok


### Variables we need to calculate percentages later

In [5]:
### need to calculate percentages: population 25 and older...
##### population age 16 and older in labor force, population for whom poverty status determined
comparison_vars = []

# people over age 25:
print('finding variables for men of any race by ages 25 and over...')
comparison_vars += metadata_df['variable'][
    (metadata_df['sequence_number'].apply(lambda x: ((x >= 2) & (x <= 2))))
    & (metadata_df['description'].apply(lambda x: 'age' in str(x).lower()))
    & (metadata_df['description'].apply(lambda x: '%male%' in str(x).lower()))
    & (metadata_df['variable'].apply(lambda x: re.search("[A-Z]_", str(x)) == None))  # note: case-sensitive search for varnames
].tolist()[8:]

print('finding variables for women of any race by ages 25 and over...')
comparison_vars += metadata_df['variable'][
    (metadata_df['sequence_number'].apply(lambda x: ((x >= 2) & (x <= 2))))
    & (metadata_df['description'].apply(lambda x: 'age' in str(x).lower()))
    & (metadata_df['description'].apply(lambda x: '%female%' in str(x).lower()))
    & (metadata_df['variable'].apply(lambda x: re.search("[A-Z]_", str(x)) == None))  # note: case-sensitive search for varnames
].tolist()[8:]

comparison_vars += ['B23025_002'] # EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND OVER%Population 16 years and over%Total%In labor force: Seq 79 in 2017
comparison_vars += ['B17001_001'] # POVERTY STATUS IN THE PAST 12 MONTHS BY SEX BY AGE% Population for whom poverty status is determined%Total: Seq 48 in 2017

# CHECK!
#metadata_df[metadata_df['variable'].apply(lambda x: x in comparison_vars)]

if (len(varlist) < len(comparison_vars)):
    varlist += comparison_vars

# CHECK!
#metadata_df[metadata_df['variable'].apply(lambda x: x in varlist)]
print('ok')


finding variables for men of any race by ages 25 and over...
finding variables for women of any race by ages 25 and over...
ok


# Parse estimates and margins of error

In [6]:
s = time.time()
states = state_codes_df.index.values.tolist()
states = [x.lower() for x in states if x not in ('AS', 'GU', 'MP', 'PR', 'UM', 'VI')]
states = ['md']

estimates_df = pandas.DataFrame()
margins_of_error_df = pandas.DataFrame()
for i in range(1, lastfilenum+1):
    if ((i == 1) | (np.mod(i,25) == 0) | (i == lastfilenum)):
        print('Processing sequence {0:,.0f}...'.format(i))
    #print('Parsing estimates and errors...')
    this_seq_estimates_df = pandas.DataFrame()
    this_seq_margins_of_error_df = pandas.DataFrame()
    for onestate in states:
        #if (i > 1):
        #print('Processing {:}...'.format(onestate))
        estimates_statefilename = rawdatadir + 'e{0:.0f}5{1:}{2:04d}000.txt'.format(thisyear, onestate, i)
        margins_of_error_statefilename = rawdatadir + 'm{0:.0f}5{1:}{2:04d}000.txt'.format(thisyear, onestate, i)
        estimates_onestate_df = pandas.read_csv(estimates_statefilename, header=None, sep=',', encoding='utf-8', low_memory=False)
        margins_of_error_onestate_df = pandas.read_csv(margins_of_error_statefilename, header=None, sep=',', encoding='utf-8', low_memory=False)
        column_names = metadata_df[metadata_df['sequence_number'] == i].set_index('variable').T.columns.tolist()
        if (i > 1):
            column_names = ['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO'] + column_names
        estimates_onestate_df.columns = column_names
        margins_of_error_onestate_df.columns = column_names
        this_seq_estimates_df = this_seq_estimates_df.append(estimates_onestate_df, sort=False)
        this_seq_margins_of_error_df = this_seq_margins_of_error_df.append(margins_of_error_onestate_df)
        if (i == 1):
            estimates_df = this_seq_estimates_df
            margins_of_error_df = this_seq_margins_of_error_df
        else:
            this_seq_estimates_df = this_seq_estimates_df.drop(['FILEID', 'FILETYPE', 'CHARITER', 'SEQUENCE'], axis=1)
            this_seq_margins_of_error_df = this_seq_margins_of_error_df.drop(['FILEID', 'FILETYPE', 'CHARITER', 'SEQUENCE'], axis=1)
            estimates_df = estimates_df.merge(this_seq_estimates_df, how='left', on=['STUSAB','LOGRECNO'], sort=False)
            margins_of_error_df = margins_of_error_df.merge(this_seq_margins_of_error_df, how='left', on=['STUSAB','LOGRECNO'], sort=False)

print('Converting all numeric columns to numeric...')
estimates_df = estimates_df.drop(['FILEID', 'FILETYPE', 'CHARITER', 'SEQUENCE'], axis=1)
margins_of_error_df = margins_of_error_df.drop(['FILEID', 'FILETYPE', 'CHARITER', 'SEQUENCE'], axis=1)

for x in estimates_df.columns[1:].tolist():
    #print('{:}...'.format(x))
    estimates_df[x] = pandas.to_numeric(estimates_df[x], errors='coerce')

for x in margins_of_error_df.columns[1:].tolist():
    #print('{:}...'.format(x))
    margins_of_error_df[x] = pandas.to_numeric(margins_of_error_df[x], errors='coerce')    

e = time.time()
print('\n')
print('Done in {0:,.0f} minutes {1:.0f} seconds!'.format((e-s)/60, (e-s) % 60))


Processing sequence 1...
Processing sequence 25...
Processing sequence 50...
Processing sequence 75...
Processing sequence 100...
Processing sequence 113...
Converting all numeric columns to numeric...


Done in 5 minutes 24 seconds!


## Get only the variables we need (see how we found them above)

### The estimates and margins of error for these variables (get the metadata later)

In [7]:
print('Getting only the variables we need...')
estimates_df = estimates_df[varlist]
margins_of_error_df = margins_of_error_df[varlist]
print('Done!')

Getting only the variables we need...
Done!


# Getting data and error values into one dataframe

In [8]:
print('renaming error columns...')
error_column_names = margins_of_error_df.columns.tolist()
error_column_names = ['STUSAB', 'LOGRECNO'] + [x+'_err' for x in error_column_names if x not in ['STUSAB', 'LOGRECNO'] and '_err' not in x]
margins_of_error_df.columns = error_column_names

#estimates_df
print('joining data and error columns...')

acs5_df = estimates_df.merge(margins_of_error_df, how='left', on=['STUSAB', 'LOGRECNO'])
acs5_df.sample(1).T



renaming error columns...
joining data and error columns...


,3191
STUSAB,md
LOGRECNO,7513
B01001_001,968
B11001_001,476
B02001_002,666
B02001_003,290
B25003_002,403
B19113_001,89152
B15002_011,164
B15002_028,55


# Get and join geography files

In [9]:
#acs5_df = estimates_df
s = time.time()
geo_df = pandas.DataFrame()

states = state_codes_df.index.values.tolist()
states = [x.lower() for x in states if x not in ('AS', 'GU', 'MP', 'PR', 'UM', 'VI')]
states = ['md']

if (debug >= 1):
        print('Reading geography files...')
for onestate in states:
    if (thisyear >= 2016):
        filename = geodir+'{0:}.xlsx'.format(onestate)
    else:
        filename = geodir+'{0:}.xls'.format(onestate)
    #if (debug == 2):
        #print('Reading geography for {0:}...'.format(onestate))
    this_geo_df = pandas.read_excel(filename)
    geo_df = pandas.concat((geo_df, this_geo_df), sort=False)

if (thisyear == 2017):
    geo_df = geo_df.assign(STUSAB = geo_df['State'].apply(lambda x: x.lower()))
else:
    geo_df = geo_df.assign(STUSAB = geo_df['STATE'].apply(lambda x: x.lower()))

geo_df = geo_df.rename(columns={'Geography ID': 'GEOID'})

#if (debug >= 1):
#    print('Retaining only tract-level geographies...')
##    print('Retaining only tract- and block-group-level geographies...')
#geo_df = geo_df[(geo_df['GEOID'].apply(lambda x: x[0:3] == '140'))]
##geo_df = geo_df[(geo_df['GEOID'].apply(lambda x: x[0:3] == '140')) | (geo_df['GEOID'].apply(lambda x: x[0:3] == '150'))]

##geo_df = geo_df.set_index('GEOID')  #We'll set GEOID as index colum AFTER the merge
if (thisyear == 2017):
    acs5_df = acs5_df.merge(geo_df, how='inner', left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'Logical Record Number'])
else:
    acs5_df = acs5_df.merge(geo_df, how='inner', left_on=['STUSAB', 'LOGRECNO'], right_on=['STUSAB', 'LOGRECNO'])
acs5_df = acs5_df.set_index('GEOID')
print('Done!')


Reading geography files...
Done!


# Write outfiles

In [10]:
acs5_df.to_csv(outfile, encoding='utf-8')
metadata_df[metadata_df['variable'].apply(lambda x: x in varlist)].to_csv(metaoutfile, encoding='utf-8')

print('Data written out to:\n{0:}\nMetadata written out to:\n{1:}'.format(outfile, metaoutfile))



Data written out to:
/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/datasets/acs5/acs5_md_2011.csv
Metadata written out to:
/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/datasets/acs5/acs5_metadata_md_2011.csv


In [ ]:

df = pandas.read_csv(outfile, index_col='GEOID')
meta_df = pandas.read_csv(metaoutfile, index_col='varnum')
df.sample(1).T

## SEQUENCE GUIDE FOR ACS 5-yr in 2017

Seq 1: unweighted count

Seq 2-3: age-sex

Seq 4: race

Seq 5: hispanic origin

Seq 6-7: ancestry

Seq 8-12: foreign birth

Seq 13-15: Place of Birth - Native

Seq 16-22: Residence Last Year - Migration

Seq 23-33: Journey to Work

Seq 34: Children - Relationship

Seq 35: Grand(Persons) - Age of HH Members

Seq 36-37: Households - Families

Seq 38-39: Marital Status

Seq 40: Fertility

Seq 41-42: School Enrollment

Seq 43-44: Educational Attainment

Seq 45-47: Language

Seq 48-56: Poverty

Seq 57-58: Disability

Seq 59-66: Income

Seq 67-72: Earnings

Seq 73-74: Veteran Status

Seq 75: Transfer Programs

Seq 76-79: Employment Status

Seq 80-102: Industry-Occupation-Class of Worker

Seq 103-112: Housing

Seq 113-123: Group Quarters

Seq 124-127: Health Insurance

Seq 128 Computer and Internet Usage

Seq 129: Quality Measures

Seq 130-133: Imputations